In [ ]:
!nsys profile -s none -o scipy_v1 -f true --stats=true python3 icassp_scipy_v1.py float32 5

In [ ]:
!nsys profile -s none -o scipy_v1 -f true --stats=true python3 icassp_scipy_v1.py float64 5

### Single Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 3,472.3  | 3,469.0     | 1.0      |
| CuPy (Baseline)       |           |          |             |          |
| CuPy (User Cache)     |           |          |             |          |
| CuPy (Data Type)      |           |          |             |          |
| CuPy (Fast Math)      |           |          |             |          |
| CuPy (Fatbin)         |           |          |             |          |
| CuPy (Launch Bounds)  |           |          |             |          |

### Double Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 3,470.9  | 3,470.2     | 1.0      |
| CuPy (Baseline)       |           |          |             |          |
| CuPy (User Cache)     |           |          |             |          |
| CuPy (Data Type)      |           |          |             |          |
| CuPy (Fast Math)      |           |          |             |          |
| CuPy (Fatbin)         |           |          |             |          |
| CuPy (Launch Bounds)  |           |          |             |          |

In [1]:
!nsys profile -s none -o cupy_v1 -f true --stats=true python3 icassp_cupy_v1.py float32 2000 0


The target application terminated with signal 9 (SIGKILL)
Processing events...
Capturing symbol files...


In [ ]:
!nsys stats --report gpukernsum --report nvtxppsum cupy_v1.qdrep

In [ ]:
!nsys profile -s none -o cupy_v1 -f true python3 icassp_cupy_v1.py float64 2000 0

In [ ]:
!nsys stats --report gpukernsum --report nvtxppsum cupy_v1.qdrep

### Single Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 3,472.3  | 3,469.0     | 1.0      |
| CuPy (Baseline)       |           | 401.1    | 0.911       | 3807.9   |
| CuPy (User Cache)     |           |          |             |          |
| CuPy (Data Type)      |           |          |             |          |
| CuPy (Fast Math)      |           |          |             |          |
| CuPy (Fatbin)         |           |          |             |          |
| CuPy (Launch Bounds)  |           |          |             |          |

### Double Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 3,470.9  | 3,470.2     | 1.0      |
| CuPy (Baseline)       |           |          |             |          |
| CuPy (User Cache)     |           |          |             |          |
| CuPy (Data Type)      |           |          |             |          |
| CuPy (Fast Math)      |           |          |             |          |
| CuPy (Fatbin)         |           |          |             |          |
| CuPy (Launch Bounds)  |           |          |             |          |

In [ ]:
!nsys profile -s none -o cupy_v2 -f true python3 icassp_cupy_v2.py float32 2000 0

In [ ]:
!nsys stats --report gpukernsum --report nvtxppsum cupy_v2.qdrep

In [ ]:
!nsys profile -s none -o cupy_v2 -f true python3 icassp_cupy_v2.py float64 2000 0

In [ ]:
!nsys stats --report gpukernsum --report nvtxppsum cupy_v2.qdrep

### Single Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 2,140    | 2,139       | 1        |
| Numba (Baseline)      |||||
| Numba (User Cache)    |||||
| Numba (Data Type)     |||||
| Numba (Fast Math)     |||||
| Numba (Max Registers) |||||

### Double Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 2,158    | 2,153       | 1        |
| Numba (Baseline)      |||||
| Numba (User Cache)    |||||
| Numba (Data Type)     |||||
| Numba (Fast Math)     |||||
| Numba (Max Registers) |||||

In [ ]:
!nsys profile -s none -o cupy_v3 -f true python3 icassp_cupy_v3.py float32 2000 0

In [ ]:
!nsys stats --report gpukernsum --report nvtxppsum cupy_v3.qdrep

In [ ]:
!nsys profile -s none -o cupy_v3 -f true python3 icassp_cupy_v3.py float64 2000 0

In [ ]:
!nsys stats --report gpukernsum --report nvtxppsum cupy_v3.qdrep

### Single Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 2,140    | 2,139       | 1        |
| Numba (Baseline)      |||||
| Numba (User Cache)    |||||
| Numba (Data Type)     |||||
| Numba (Fast Math)     |||||
| Numba (Max Registers) |||||

### Double Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 2,158    | 2,153       | 1        |
| Numba (Baseline)      |||||
| Numba (User Cache)    |||||
| Numba (Data Type)     |||||
| Numba (Fast Math)     |||||
| Numba (Max Registers) |||||

In [ ]:
!nsys profile -s none -o cupy_v4 -f true python3 icassp_cupy_v4.py float32 2000 0

In [ ]:
!nsys stats --report gpukernsum --report nvtxppsum cupy_v4.qdrep

In [ ]:
!nsys profile -s none -o cupy_v4 -f true python3 icassp_cupy_v4.py float64 2000 0

In [ ]:
!nsys stats --report gpukernsum --report nvtxppsum cupy_v4.qdrep

### Single Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 2,140    | 2,139       | 1        |
| Numba (Baseline)      |||||
| Numba (User Cache)    |||||
| Numba (Data Type)     |||||
| Numba (Fast Math)     |||||
| Numba (Max Registers) |||||

### Double Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 2,158    | 2,153       | 1        |
| Numba (Baseline)      |||||
| Numba (User Cache)    |||||
| Numba (Data Type)     |||||
| Numba (Fast Math)     |||||
| Numba (Max Registers) |||||

In [ ]:
!nvcc --fatbin -std=c++11 --use_fast_math \
    --generate-code arch=compute_70,code=sm_70 \
    --generate-code arch=compute_75,code=sm_75 \
    --generate-code arch=compute_80,code=[sm_80,compute_80] \
    _lombscargle.cu -odir .

In [ ]:
!nsys profile -s none -o cupy_v5 -f true python3 icassp_cupy_v5.py float32 2000 0

In [ ]:
!nsys stats --report gpukernsum --report nvtxppsum cupy_v5.qdrep

In [ ]:
!nsys profile -s none -o cupy_v5 -f true python3 icassp_cupy_v5.py float64 2000 0

In [ ]:
!nsys stats --report gpukernsum --report nvtxppsum cupy_v5.qdrep

### Single Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 2,140    | 2,139       | 1        |
| Numba (Baseline)      |||||
| Numba (User Cache)    |||||
| Numba (Data Type)     |||||
| Numba (Fast Math)     |||||
| Numba (Max Registers) |||||

### Double Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 2,158    | 2,153       | 1        |
| Numba (Baseline)      |||||
| Numba (User Cache)    |||||
| Numba (Data Type)     |||||
| Numba (Fast Math)     |||||
| Numba (Max Registers) |||||

In [ ]:
!nvcc --fatbin -std=c++11 --use_fast_math \
    --generate-code arch=compute_70,code=sm_70 \
    --generate-code arch=compute_75,code=sm_75 \
    --generate-code arch=compute_80,code=[sm_80,compute_80] \
    _lombscargle_lb.cu -odir .

In [ ]:
!nsys profile -s none -o cupy_v6 -f true python3 icassp_cupy_v6.py float32 2000 0

In [ ]:
!nsys stats --report gpukernsum --report nvtxppsum cupy_v6.qdrep

In [ ]:
!nsys profile -s none -o cupy_v6 -f true python3 icassp_cupy_v6.py float64 2000 0

In [ ]:
!nsys stats --report gpukernsum --report nvtxppsum cupy_v6.qdrep

### Single Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 2,140    | 2,139       | 1        |
| Numba (Baseline)      |||||
| Numba (User Cache)    |||||
| Numba (Data Type)     |||||
| Numba (Fast Math)     |||||
| Numba (Max Registers) |||||

### Double Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 2,158    | 2,153       | 1        |
| Numba (Baseline)      |||||
| Numba (User Cache)    |||||
| Numba (Data Type)     |||||
| Numba (Fast Math)     |||||
| Numba (Max Registers) |||||